In [1]:
import os
import time
import torch
import sys
import torch.optim as optim
import numpy as np
from torch.autograd import Variable
from PIL import Image
from torchvision import transforms as T
from matplotlib import pyplot as plt
%matplotlib notebook

In [2]:
from models import SpecialFuseNetModel
from data_manager import rgbd_gradients_dataset, rgbd_gradients_dataloader
from train import FuseNetTrainer
from hyperparameters import *

In [3]:
CWD             = os.getcwd()
# DATASET_DIR     = os.path.join(CWD,'data/nyuv2')
DATASET_DIR     = os.path.join(CWD,'data_overfit/nyuv2')
print(DATASET_DIR)

C:\Users\tomav\Documents\GitHub\cs236781-project\data_overfit/nyuv2


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [5]:
rgbd_grads_ds = rgbd_gradients_dataset(root=DATASET_DIR, use_transforms=True)

In [6]:
dl_train,dl_test = rgbd_gradients_dataloader(root=DATASET_DIR,
                                             use_transforms=True)

In [7]:
# _ = plot.rgbd_gradients_dataset_first_n(dataset=rgbd_grads_ds,n=5)
print(f'Found {len(rgbd_grads_ds)} images in dataset folder.')

Found 2 images in dataset folder.


In [8]:
sample_batch = next(iter(dl_train))
rgb_size = tuple(sample_batch['rgb'].shape[1:])
depth_size = tuple(sample_batch['depth'].shape[1:])
grads_size = tuple(sample_batch['x'].shape[1:])

In [9]:
fusenetmodel = SpecialFuseNetModel(rgb_size=rgb_size,depth_size=depth_size,grads_size=grads_size, device=device)

[I] - default optimizer set: SGD(lr=0.001,momentum=0.9,weight_decay=0.0005)
[I] - default scheduler set: StepSR(step_size=1000,gamma=0.1)


In [10]:
# trainer = FuseNetTrainer(model=fusenetmodel, device=device)

In [11]:
checkpoint_file = 'checkpoints/special_fusenet_0'
# if os.path.isfile(f'{checkpoint_file}.pt'):
#     os.remove(f'{checkpoint_file}.pt')

In [12]:
# res = trainer.fit(dl_train, dl_test, early_stopping=400, print_every=10,
#                   checkpoints=checkpoint_file)

In [13]:
fusenetmodel.load_state_dict(torch.load(checkpoint_file + '.pt', map_location=device), strict=False)

img_rgb = Image.open(DATASET_DIR + "/rgb/0.png")
img_depth = Image.open(DATASET_DIR + "/depth/0.png")
img_rgb_tensor, img_depth_tensor = rgbd_grads_ds.transform(img_rgb, img_depth)
img_rgb_tensor.unsqueeze_(0)
img_depth_tensor.unsqueeze_(0)
output = fusenetmodel(Variable(img_rgb_tensor), Variable(img_depth_tensor))

output = output.cpu().detach().numpy()[0]
print("x")
print(output[0])
print("x sum:")
print(np.sum(output[0]))
print("y")
print(output[1])
print("y sum:")
print(np.sum(output[1]))

x
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
x sum:
0.0
y
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
y sum:
0.0
